# SOLUTION

### What drives the price of a car?

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

---
### 1_ Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 
---
---

**Objetivo**: Determinar los factores que influyen en el precio de los autos usados y dar las recomendaciones claras a un concesionario para optimizar su oferta.

**Preguntas clave a idenfiticar**:
- ¿Qué características hacen que un auto sea más o menos valioso?
- ¿Qué atributos buscan los consumidores en un auto usado?
- ¿Cómo puede el concesionario ajustar precios en base a los hallazgos?

**Fuente de datos inicial**: Conjunto de datos reducido (426,000 registros) extraído de un conjunto original de 3 millones de autos usados (Kaggle).

**Atributos importantes**:
- Marca
- Modelo
- Año
- Kilometraje
- Tipo de combustible
- Transmisión
- Número de puertas
- Potencia del motor

**Variable objetivo**: Precio


---
### 2_ Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to nform your business understanding.
---
---

In [ ]:
import pandas as pd

# Cargar el archivo CSV para revisar el contenido
file_path = './data/vehicles.csv'
vehicles_data = pd.read_csv(file_path)

# Mostrar información general y primeras filas
vehicles_data_info = vehicles_data.info()
vehicles_data_head = vehicles_data.head()

vehicles_data_info, vehicles_data_head

In [ ]:
import pandas as pd

# Cargar el archivo CSV para revisar el contenido
file_path = './data/vehicles.csv'
vehicles_data = pd.read_csv(file_path)

# Mostrar información general y primeras filas
vehicles_data_info = vehicles_data.info()
vehicles_data_head = vehicles_data.head()

vehicles_data_info, vehicles_data_head

![](images/img2.png)

### Descripción Inicial del Dataset

## Tamaño
- **426,880 filas** y **18 columnas**.

### Columnas principales
- **price**: Precio del vehículo (variable objetivo).
- **year**: Año de fabricación (muchos valores nulos).
- **manufacturer, model, condition, cylinders, fuel, etc.**: Factores potenciales que pueden influir en el precio.
- **odometer**: Distancia recorrida por el vehículo.
- **title_status, transmission, drive, type, paint_color**: Factores categóricos con valores faltantes significativos.
- **VIN**: Número de identificación del vehículo, probablemente no sea relevante para el análisis.

### Observaciones
- Hay valores faltantes en varias columnas, particularmente en `condition`, `cylinders`, `size` y `type`.
- Algunas columnas como `region` o `VIN` podrían no ser útiles para el análisis predictivo.

### >>> Próximos pasos <<<

### Limpieza inicial
1. Identificar valores faltantes críticos y eliminar columnas irrelevantes.
2. Tratar valores extremos en el precio.

### Exploración de datos
1. Examinar la distribución del precio y su relación con otras variables.
2. Generar visualizaciones para detectar tendencias importantes.

### Preparación para el modelo
1. Codificación de variables categóricas.
2. Dividir datos en conjuntos de entrenamiento y prueba.


---
### 3_ Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 
---
---

### >>> Plan de Limpieza de Datos <<<

### General

### Valores Nulos
- **Variables críticas** (`year`, `manufacturer`, `model`, `fuel`, `odometer`, etc.):
  - Evaluar imputación con medidas estadísticas como:
    - Media, mediana o moda.
  - Eliminar filas si los valores ausentes son críticos para el análisis.
- **Columnas con demasiados valores faltantes**:
  - Eliminar columnas como `size` si no aportan información valiosa al modelo.

### Columnas Irrelevantes
- **Eliminar columnas** que no impacten el análisis:
  - Ejemplo: `id`, `VIN`, `region`.

---

### >>> Limpieza del Campo `price`

### Identificar Precios No Razonables
- Filtrar valores extremos:
  - Valores extremadamente bajos (e.g., `0` o `1` USD).
  - Valores excesivamente altos (e.g., millones de USD).
- Basarse en:
  - Estadísticas descriptivas.
  - Valores esperados del mercado de autos usados.

### Acción
- Eliminar registros con precios fuera de un rango razonable para evitar sesgos en el modelo.

---

### >>> Datos Categóricos

### Codificación
- Convertir variables categóricas en variables numéricas:
  - Ejemplo: `condition`, `fuel`, `transmission`, `drive`, `type`, `paint_color`.
  - Métodos: One-Hot Encoding o Label Encoding.

### Unificación de Valores
- Detectar y corregir inconsistencias:
  - Formatos diferentes o errores tipográficos (e.g., "Diesel" vs "diesel").

---

### Outliers (Datos Atípicos)

### Identificación
- Detectar valores extremos en variables continuas como:
  - `odometer` y `year`.
- Métodos:
  - Rango intercuartílico (IQR).
  - Z-score.

### Tratamiento
- Decidir si:
  - Eliminar outliers.
  - Transformar valores extremos según el contexto del análisis.


In [ ]:
# Paso 1: Eliminar columnas irrelevantes
irrelevant_columns = ['id', 'VIN', 'region']
vehicles_data_cleaned = vehicles_data.drop(columns=irrelevant_columns)

# Paso 2: Filtrar precios no razonables (ejemplo: precios <= 100 USD o mayores a 250,000 USD)
reasonable_price_range = (vehicles_data_cleaned['price'] > 100) & (vehicles_data_cleaned['price'] <= 250000)
vehicles_data_cleaned = vehicles_data_cleaned[reasonable_price_range]

# Paso 3: Revisar valores nulos y eliminarlos en variables críticas
critical_columns = ['price', 'year', 'manufacturer', 'model', 'fuel', 'odometer']
vehicles_data_cleaned = vehicles_data_cleaned.dropna(subset=critical_columns)

# Resumen después de la limpieza inicial
cleaned_info = vehicles_data_cleaned.info()
cleaned_stats = vehicles_data_cleaned.describe(include='all')

cleaned_info, cleaned_stats


![](images/img2a.png)

### >>> Resultados de la Limpieza Inicial

### Datos Filtrados
- Reducción a **365,951 registros** al eliminar precios fuera del rango razonable (USD 100 a USD 250,000) y filas con valores críticos faltantes.

### Características Principales
- **price**: Rango entre USD 101 y USD 244,999.
- **year**: Desde 1900 hasta 2022, pero la mayoría de los autos están entre 2008 y 2017 (mediana: 2013).
- **manufacturer**: 41 fabricantes únicos, liderados por Ford.
- **fuel**: Cinco tipos, con predominio de gasolina.
- **odometer**: Varía entre 0 y 10,000,000 millas; mediana alrededor de 87,159.

### Columnas con Datos Faltantes
- **Columnas con valores nulos significativos**: 
  - `condition`, `cylinders`, `drive`, `size`, `type`, y `paint_color`.
  - A pesar de los valores faltantes, estas columnas podrían seguir siendo útiles.

---

### >>> Próximos Pasos

### Imputar o Tratar Valores Faltantes
- **Imputación**:
  - Usar moda, mediana o crear una categoría "desconocido" para columnas categóricas como `condition` y `cylinders`.

### Visualización de Datos
- **Exploración de relaciones clave**:
  - Generar gráficos para analizar la relación entre `price` y variables como:
    - `year`
    - `manufacturer`
    - `odometer`
    - `fuel`

### Preparación para Modelado
- **Codificación**:
  - Convertir variables categóricas en variables numéricas.
- **Normalización**:
  - Escalar valores continuos como `year` y `odometer`.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Imputación básica de valores faltantes en columnas categóricas
categorical_columns = ['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color']
vehicles_data_cleaned[categorical_columns] = vehicles_data_cleaned[categorical_columns].fillna('unknown')

# Paso 2: Gráficos exploratorios
# Relación entre precio y año de fabricación
plt.figure(figsize=(12, 6))
sns.boxplot(data=vehicles_data_cleaned, x='year', y='price')
plt.title('Distribución del precio según el año de fabricación', fontsize=14)
plt.xticks(rotation=45)
plt.xlabel('Año de fabricación')
plt.ylabel('Precio (USD)')
plt.tight_layout()
plt.show()

# Relación entre precio y combustible
plt.figure(figsize=(8, 6))
sns.boxplot(data=vehicles_data_cleaned, x='fuel', y='price')
plt.title('Distribución del precio según el tipo de combustible', fontsize=14)
plt.xlabel('Combustible')
plt.ylabel('Precio (USD)')
plt.tight_layout()
plt.show()

# Relación entre precio y condición
plt.figure(figsize=(8, 6))
sns.boxplot(data=vehicles_data_cleaned, x='condition', y='price')
plt.title('Distribución del precio según la condición del vehículo', fontsize=14)
plt.xlabel('Condición')
plt.ylabel('Precio (USD)')
plt.tight_layout()
plt.show()

# Relación entre precio y odómetro
plt.figure(figsize=(10, 6))
sns.scatterplot(data=vehicles_data_cleaned, x='odometer', y='price', alpha=0.3)
plt.title('Relación entre odómetro y precio', fontsize=14)
plt.xlabel('Odómetro (millas)')
plt.ylabel('Precio (USD)')
plt.tight_layout()
plt.show()


### >>> Visualización de los Datos

### Precio vs. Año de Fabricación
- Los vehículos más recientes (después de 2015) tienden a tener **precios más altos**, como era de esperar.
- Hay una **dispersión significativa** en los vehículos más antiguos, reflejando el impacto del estado y otros factores.

### Precio vs. Tipo de Combustible
- **Autos eléctricos** tienen un precio promedio más alto.
- **Gasolina** es el tipo de combustible más común, con una amplia gama de precios.

### Precio vs. Condición del Vehículo
- Vehículos en **excelente condición** tienden a ser más costosos.
- Vehículos en condiciones **justas** o **desconocidas** tienen precios más bajos.

### Precio vs. Odómetro
- Se observa una **tendencia negativa**: los autos con mayor millaje tienden a tener precios más bajos.
- Existen **valores atípicos**, como autos con alto millaje y precios elevados, que podrían ser modelos de lujo o restaurados.

---

### Próximos Pasos

### Preparar los Datos para el Modelo
- **Codificar variables categóricas.**
- **Escalar variables continuas** como `odometer` y `year`.

### Construir un Modelo de Machine Learning
- Usar un **modelo de regresión** para predecir el precio.
- Evaluar la **importancia de las características**.


### >>> Preparación de los Datos para el Modelo

### >>> Pasos a seguir

### 1. Codificación de Variables Categóricas
- Transformar variables como `manufacturer`, `fuel`, `condition`, etc., en un formato numérico mediante **One-Hot Encoding**.

### 2. Escalado y Normalización
- **Normalizar** variables continuas como `year` y `odometer` para facilitar el aprendizaje de los modelos.

### 3. División de Datos
- Separar los datos en conjuntos de **entrenamiento** y **prueba** con una proporción de **70/30**.

---
### 4_ Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.
---
---

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Separar características (X) y variable objetivo (y)
X = vehicles_data_cleaned.drop(columns=['price'])
y = vehicles_data_cleaned['price']

# Identificar columnas categóricas y numéricas
categorical_features = ['year', 'manufacturer', 'fuel', 'condition', 'drive', 'type', 'paint_color']
numerical_features = ['odometer']

# Paso 2: Preprocesamiento de los datos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Paso 3: Crear pipeline con modelo RandomForestRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Paso 4: Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 5: Búsqueda en cuadrícula para optimización de hiperparámetros
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20, None]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados por la búsqueda en cuadrícula
print(f"Mejores parámetros: {grid_search.best_params_}")

# Paso 6: Evaluación del modelo con los mejores parámetros
best_model = grid_search.best_estimator_

# Predicciones sobre el conjunto de prueba
y_pred = best_model.predict(X_test)

# Cálculo de métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Mostrar métricas
print(f"MSE: {mse}")
print(f"R²: {r2}")
print(f"RMSE: {rmse}")

# Paso 7: Importancia de las características
feature_importances = best_model.named_steps['regressor'].feature_importances_
print(f"Importancia de las características: {feature_importances}")


![](images/img1.png)

**Nota: Este calculo tomo aproximadamente 1 hora con 3 minutos.**

---
### 5_ Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.
---
---

Una vez que que se completo el proceso de modelado, el siguiente paso es reflexionar sobre la calidad del modelo y lo que se ha aprendido de los resultados obtenidos. Para ello, se ha utilizado varias métricas de evaluación, como el **MSE**, **R²** y **RMSE**.

- **MSE (Error cuadrático medio):** El modelo presenta un MSE de 33,498,131.27, lo que sugiere que hay un error promedio en las predicciones. Aunque este valor es relativamente alto, indica que el modelo podría mejorar, ya que es común que los modelos iniciales tengan este tipo de error.
  
- **R² (Coeficiente de determinación):** El valor de R² es **0.845**, lo que significa que el modelo explica aproximadamente el **84.5%** de la variabilidad en los precios de los autos. Este es un buen indicador de que el modelo tiene un ajuste razonable a los datos.

- **RMSE (Raíz del error cuadrático medio):** El RMSE de **5,787.76** indica que la desviación estándar entre las predicciones y los valores reales es de aproximadamente 5,787.76 unidades monetarias. Aunque es un valor relativamente alto, es importante tener en cuenta que el precio de los autos usados varía significativamente.

A pesar de los resultados satisfactorios en términos de **R²**, también se ha observado que algunas características clave como el **año** y **kilometraje** no tienen una fuerte correlación con el **precio** de los autos. Esto puede sugerir que otras variables, como la **marca**, **modelo**, o **estado del vehículo**, pueden estar influyendo más en los precios de los autos. **El modelo de Random Forest** ha demostrado ser efectivo en términos de precisión, pero se recomienda seguir ajustándolo o incluir nuevas características para mejorar aún más los resultados.

**Recomendación de mejora:** Realizar un análisis más profundo para incluir más variables relevantes o probar otros modelos para comparar y verificar si el modelo actual es el óptimo para predecir el precio de los autos.

---
### 6_ Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.
---
---

Una vez que se ha definido y evaluado el modelo, se entrega la información y recomendaciones al cliente. Este reporte debe ser comprensible y útil para los concesionarios de autos usados, ya que les ayudará a tomar decisiones basadas en el análisis de los datos.

**Hallazgos principales del modelo:**
1. **Factores determinantes del precio:** Los autos más **nuevos** y con **menos kilometraje** tienden a tener precios más altos. Sin embargo, en este conjunto de datos, el impacto del **año** y el **kilometraje** es relativamente bajo, lo que podría indicar que otros factores como **modelo**, **marca**, **condición** y **tipo de combustible** son más determinantes.
   
2. **Modelo predictivo utilizado:** El modelo **RandomForestRegressor** ha demostrado ser el más preciso en cuanto a predicción de precios, con un **R² de 0.845** y un **MSE** de 33,498,131.27. Este modelo debería ser la base para predecir precios de autos usados, pero se recomienda seguir mejorando con más datos y características.

3. **Recomendaciones de inventario:**
   - Priorizar autos con **menos kilometraje** y **modelos más recientes**, ya que estos son más valorados por los consumidores. Debería hacerse un análisis continuo para ajustar el inventario de acuerdo con la oferta y la demanda del mercado.
   - Considerar otros factores clave como el **modelo**, **marca**, **tipo de combustible** y **estado general del vehículo**, ya que estos pueden tener una mayor influencia en el precio que el año o kilometraje.

4. **Acción futura a seguir:**
   - **Monitoreo constante:** Es esencial seguir monitoreando los precios de los autos usados en el mercado, actualizando el modelo de predicción conforme evolucionen los patrones de compra y venta.
   - **Refinamiento del modelo:** Incluir más características en el modelo, como la **marca**, **estado** del auto, **color**, **transmisión**, etc., podría mejorar aún más la precisión de las predicciones. 

Por lo que este informe proporciona una base sólida para que los concesionarios de autos usados ajusten su inventario, priorizando autos con características que generan mayor valor para los consumidores. Además, el modelo puede seguir evolucionando para adaptarse a cambios en las tendencias del mercado y optimizar la toma de decisiones comerciales.
